
# Reproject ICESat-2 Elevation Points with GeoPandas

This notebook illustrates reprojecting (Lon,Lat,Elevation) points in one 3D CRS to another. 

:::{note} Learning Goals
- See how to set 3D Geometries in GeoPandas when working with 3D CRS
- Use PROJ to examine reprojection options
- Reproject points between specific ITRF and WGS realizations (ITRF2014 to WGS 84 (G2139))
- Use tests and view logs to ensure your reprojection is successful and valid
:::

## Import libraries and configure logging

:::{tip}
Libraries like GeoPandas use GDAL and PROJ behind the scenes to perform reprojection between different CRS. Sometimes, there are different algorithms that may be used for reprojection. It can be helpful to expose logging messages to see which libraries and which algorithms are being used behind the scenes.
:::

In [ ]:
# This environment variable must be set before importing geopandas for logging
import os
os.environ['PROJ_DEBUG'] = '2'
# Ensure this is 'ON' to get shift grids over the internet
print(os.environ['PROJ_NETWORK'])

import logging

In [ ]:
import fiona
import geopandas as gpd

In [ ]:
# It's good to keep track of versions of geospatial libraries and dependencies
gpd.show_versions()

## Load 3D points

In [ ]:
# ICESat-2 data saved from sliderule:
#gf = icesat2.atl06p({}, resources=['ATL03_20181019224323_03250112_005_01.h5'])
#gf[:100].to_file('ATL03_20181019224323_03250112_005_01.geojson', driver='GeoJSON')

gf = gpd.read_file('ATL03_20181019224323_03250112_005_01.geojson')

In [ ]:
gf.head(2)

In [ ]:
gf.crs

:::{important}
Always check the location of your original data before reprojecting
:::

In [ ]:
points = gf.reset_index()
points.loc[:, 'time'] = points.time.dt.strftime('%Y-%m-%d')
points.explore(zoom_start=2, column='h_mean')

In [ ]:
# Get bounding box of all of our points
w,s,e,n = gf.union_all().bounds #W, S, E, N
print(w,s,e,n)

## Check reprojection options

The `projinfo` command is very helpful to see which algorithms (or 'pipelines') could be used to go from one CRS to another. Below we see `Candidate operations found: 75` indicating there are a total of 75 options, which are ordered top to bottom in decreasing preference! 

If logging is enabled (PROJ_DEBUG=2) you will see many lines like `pj_open_lib(us_noaa_FL.tif)` which correspond to PROJ checking for availability of shift grids required for *any* of the  possible transforms. These may either be files in local directories, or retrieved over the network from https://cdn.proj.org

In [ ]:
!PROJ_DEBUG=0 projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --hide-ballpark --spatial-test intersects | grep Candidate

:::{note}
All CRS have a "valid area", for example, some regional CRS definitions are only valid for specific countries or continents, but global CRSs (like EPSG:7912) are valid for the entire globe! This is a common reason for `projinfo` returning a lot of possible transforms. Importantly `projinfo` does not know where our data is unless we pass a `--bbox`, which can be helpful for narrowing in on the best reprojection pipeline to use.
:::

In [ ]:
!projinfo -s EPSG:7912 -t EPSG:9518 -o PROJ --grid-check none --bbox {w},{s},{e},{n}  --hide-ballpark --spatial-test intersects | head -n 20

## Reproject data 

By default geopandas will use the first operation reported by `projinfo`. In this case:

```
+proj=pipeline
  +step +proj=axisswap +order=2,1
  +step +proj=unitconvert +xy_in=deg +xy_out=rad
  +step +inv +proj=vgridshift +grids=us_nga_egm08_25.tif +multiplier=1
  +step +proj=unitconvert +xy_in=rad +xy_out=deg
  +step +proj=axisswap +order=2,1
```

☝️ `+proj=vgridshift +grids=us_nga_egm08_25.tif` will apply interpolated vertical offsets corresponding to the repojection of ellipsoid height to geoid height. For the above transform we do not have *horizontal* position changes, only vertical.

:::{important}
For Geopandas to select this PROJ pipeline, the geometry column *must be 3D* (contain elevation as a coordinate)
:::

In [ ]:
points3D = gpd.points_from_xy(gf.geometry.x, gf.geometry.y, gf.h_mean)
gf3D = gpd.GeoDataFrame(geometry=points3D, crs='EPSG:7912')

In [ ]:
gf3D[:1].get_coordinates(include_z=True)

In [ ]:
# Reprojection happens here
gfGeoid = gf3D.to_crs(epsg=9518)
gfGeoid[:1].get_coordinates(include_z=True)

### Validate results 

Once you're satisfied that the results reflect the expected magnitude of difference, it can be a good idea to add tests to your code to ensure there are no issues in the future. A common gotcha is if you do not have a vertical shift grid locally and there are network connectivity issues the reproject may not actually result in reprojected values!

In [ ]:
# X&Y coordinates should be the same, but Z should be different
gpd.pd.testing.assert_frame_equal(gfGeoid.get_coordinates(), gf3D.get_coordinates())

max_dz = (gfGeoid.get_coordinates(include_z=True).z - gf3D.get_coordinates(include_z=True).z).max().astype('int16')
assert max_dz == 29

## Avoiding bogus reprojection

:::{warning}
GeoPandas will happily do bogus transforms for you and not report error messages. Exercise caution by checking that logs show your intended transform is used, or add validation to your codebase to ensure reprojections are consistent with what you expect. This is especially important when converting data from a globally-defined CRS (because the valid extent of a global CRS always intersects the valid extent of any other CRS target extent)
:::

In [ ]:
# Here we'll use a target CRS valid only for the United States, but our data is in Antarctica!
!projinfo EPSG:2927+5703 -o WKT2:2019 --single-line

In [ ]:
# That is a complicated transform!
!PROJ_DEBUG=0 projinfo -s EPSG:7912 -t EPSG:2927+5703 -q -o PROJ

In [ ]:
logging.basicConfig(level=logging.DEBUG)

with fiona.Env(CPL_DEBUG=True):
    bogus = gf3D.to_crs(epsg="2927+5703")

:::{note}
Examining the logging messages shows which shift grids are successful located and the last line ("Using coordinate operation...") displays which reprojection pipeline is actually used. Whenever in doubt it's good to examine these logs.
:::

In [ ]:
bogus.head()

:::{note}
The above transform was performed even though the points are outside the valid area of the target CRS. X and Y values have changed significantly (and fall outside the domain of the CRS), and Z values are unchanged!
:::